<a href="https://colab.research.google.com/github/BLuckoo/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import librairies & start Spark session**

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Amazon Dataset into SPark DataFrame**

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
amazon_vreviews_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
amazon_vreviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [5]:
# Create the vine_table. DataFrame
vine_review_df = amazon_vreviews_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RY01SAV7HZ8QO|          4|            0|          0|   N|                Y|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|                N|
|R149Q3B5L33NN5|          5|            0|          0|   N|                Y|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|                Y|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|                Y|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|                N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|                Y|
|R25IK0UAHWNB22|          3|            0|          0|   N|                Y|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|                Y|
| R5XVKTHL6SITI|          5|            0|          0|   N|     

In [6]:
# Remove any "na's"
clean_vine_review_df = vine_review_df.dropna()


**Filter the vine review table where total_votes count is >= 20**

In [7]:
# Filter the DataFrame

vine_review_f1_df = clean_vine_review_df.filter("total_votes>=20")
vine_review_f1_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
|R1LV778IDGRJQ6|          1|            4|         23|   N|                Y|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R2LBEAGVJA7HK1|          3|            5|         48|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R3QLX5NODD1OJ4|          1|            9|         25|   N|                Y|
|R2SOOHE0XUQON1|          1|            2|         24|   N|     

**Filter for (helpful_votes/total_votes)>=50%**

In [8]:
#Filter new DataFrame 

vine_review_f2_df = vine_review_f1_df.filter("(helpful_votes/total_votes)>=0.5")
vine_review_f2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|
|R3Q7YJX1I5PDMU|          3|           22|         22|   N|                Y|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R2QJI05ZY7DNSR|          4|           59|         73|   N|     

**Filter for Paid reviews which were >= 50% helpful**

In [9]:
paid_vine_review_df = vine_review_f2_df.filter("vine== 'Y'")
paid_vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LLICBKRHLRJC|          4|           17|         22|   Y|                N|
| RUD135S06DEVE|          5|           41|         51|   Y|                N|
|R2WG8UL4SGDPDH|          4|           23|         25|   Y|                N|
|R158AOKOA28WS2|          4|           18|         28|   Y|                N|
| RGWV3QLJQXU9X|          5|          332|        380|   Y|                N|
| RD0E1RZSSYGGN|          5|          261|        292|   Y|                N|
|R3LGCTQH5ZMGO0|          5|           61|         68|   Y|                N|
|R2H24YTRLARCZJ|          4|          277|        297|   Y|                N|
| RYYTC9JTG7WPW|          5|          107|        134|   Y|                N|
| RTB8H4YQYTLY3|          4|           58|         62|   Y|     

**Filter for Unpaid reviews which were >= 50% helpful**

In [10]:
unpaid_vine_review_df = vine_review_f2_df.filter("vine == 'N'")
unpaid_vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|
|R3Q7YJX1I5PDMU|          3|           22|         22|   N|                Y|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R2QJI05ZY7DNSR|          4|           59|         73|   N|     

Filter for 5-star ratings within the dataset where helpful votes/(total votes over 20) is >=50%

In [11]:
# Filter 5-star ratings within all reviews

total_5star_reviews_df = vine_review_f2_df.filter("star_rating == '5'")
total_5star_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R14QWM0LJO6313|          5|           24|         28|   N|                Y|
|R19H4AKN005KL8|          5|           19|         28|   N|                Y|
|R27MJ6EZADWWPX|          5|           34|         37|   N|                Y|
|R2GFDC3KTYASPJ|          5|           94|        118|   N|                Y|
|R25PC9R5T29WSK|          5|           21|         23|   N|                Y|
|R22UO302E0W4DK|          5|           17|         23|   N|                N|
| RDCFYG43XCPNM|          5|           46|         55|   N|     

In [12]:
# Filter 5-star ratings within paid reviews

paid_5star_reviews_df = paid_vine_review_df.filter("star_rating == '5'")
paid_5star_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RUD135S06DEVE|          5|           41|         51|   Y|                N|
| RGWV3QLJQXU9X|          5|          332|        380|   Y|                N|
| RD0E1RZSSYGGN|          5|          261|        292|   Y|                N|
|R3LGCTQH5ZMGO0|          5|           61|         68|   Y|                N|
| RYYTC9JTG7WPW|          5|          107|        134|   Y|                N|
| RMQYIMBVK7T8R|          5|           29|         36|   Y|                N|
| R9KIBRBGB1M6W|          5|           26|         33|   Y|                N|
|R122J9QGFCZTR0|          5|          324|        342|   Y|                N|
|R1KG6E0W0Q5YCQ|          5|           27|         32|   Y|                N|
|R3B6CGFNVWRWJ0|          5|           15|         23|   Y|     

In [13]:
# Filter 5-star ratings within unpaid reviews

unpaid_5star_reviews_df = unpaid_vine_review_df.filter("star_rating == '5'")
unpaid_5star_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R14QWM0LJO6313|          5|           24|         28|   N|                Y|
|R19H4AKN005KL8|          5|           19|         28|   N|                Y|
|R27MJ6EZADWWPX|          5|           34|         37|   N|                Y|
|R2GFDC3KTYASPJ|          5|           94|        118|   N|                Y|
|R25PC9R5T29WSK|          5|           21|         23|   N|                Y|
|R22UO302E0W4DK|          5|           17|         23|   N|                N|
| RDCFYG43XCPNM|          5|           46|         55|   N|     

Calculations:
Total number of reviews, number of 5-star reviews, % of 5-star reviews for paid and unpaid reviews **bold text**

**All reviews**

In [14]:
# Total number of reviews
total_reviews_count = vine_review_f2_df.count()
total_reviews_count

24301

In [15]:
# Number of 5 star ratings in all reviews
total_5star_count = total_5star_reviews_df.count()
total_5star_count

11005

In [16]:
# Percentage of total 5 star_ratings in all reviews
total_percentage_5star_reviews = (total_5star_count/total_reviews_count)*100
round(total_percentage_5star_reviews,2)

45.29

**Paid Reviews**

In [17]:
# Total number of paid reviews
paid_reviews_count = paid_vine_review_df.count()
paid_reviews_count

261

In [18]:
# Number of 5 star ratings in paid reviews
paid_5star_count = paid_5star_reviews_df.count()
paid_5star_count

106

In [19]:
# Percentage of 5 star_ratings in paid reviews
paid_percentage_5star_reviews = (paid_5star_count/paid_reviews_count)*100
round(paid_percentage_5star_reviews,2)

40.61

**Unpaid Reviews**

In [20]:
# Total number of unpaid reviews
unpaid_reviews_count = unpaid_vine_review_df.count()
unpaid_reviews_count

24040

In [21]:
# Number of 5 star ratings in unpaid reviews
unpaid_5star_count = unpaid_5star_reviews_df.count()
unpaid_5star_count

10899

In [22]:
# Percentage of 5 star_ratings in unpaid reviews
unpaid_percentage_5star_reviews = (unpaid_5star_count/unpaid_reviews_count)*100
round(unpaid_percentage_5star_reviews,2)

45.34

**Statistical Analysis**

In [23]:
# Statistical analysis of raw data
clean_vine_review_df.describe().show()

+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|      total_votes|  vine|verified_purchase|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|  count|        705889|            705889|            705889|           705889|705889|           705889|
|   mean|          null|3.9021234216711127| 3.612317234012713|4.700149740256612|  null|             null|
| stddev|          null|1.4551018223137497|37.447379580656175|40.33904215089728|  null|             null|
|    min|R100044O1POV5X|                 1|                 0|                0|     N|                N|
|    max|  RZZZQPVDV7RK|                 5|             10020|            10462|     Y|                Y|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+



In [24]:
# Statistical analysis of filtered data (total_votes >=20 and helpful_votes/total_votes >=50%)
vine_review_f2_df.describe().show()

+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|  count|         24301|             24301|             24301|             24301|24301|            24301|
|   mean|          null| 3.617464301880581|  64.8849018558907| 72.08835027365129| null|             null|
| stddev|          null|1.5778553469492629|191.36097222303084|203.54740434431235| null|             null|
|    min|R100GMAY1ZSE8B|                 1|                10|                20|    N|                N|
|    max| RZZANCVZNVUP4|                 5|             10020|             10462|    Y|                Y|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+



In [25]:
# Statistical analysis of paid reviews from the filtered data (total_votes >=20 and helpful_votes/total_votes >=50%)
paid_vine_review_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|  count|           261|               261|              261|               261| 261|              261|
|   mean|          null|  4.10727969348659|75.17241379310344| 84.50191570881226|null|             null|
| stddev|          null|0.9547375164498505|136.2422508929662|144.85185069636518|null|             null|
|    min|R10M9JYKC8GX2J|                 1|               14|                20|   Y|                N|
|    max| RZ9LVVFM49QBL|                 5|             1751|              1861|   Y|                N|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+



In [26]:
# Statistical analysis of unpaid reviews from the filtered data (total_votes >=20 and helpful_votes/total_votes >=50%)
unpaid_vine_review_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+-----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+-----+-----------------+
|  count|         24040|             24040|            24040|             24040|24040|            24040|
|   mean|          null| 3.612146422628952|64.77321131447587| 71.95357737104825| null|             null|
| stddev|          null|1.5824557755748934|191.8715308978137|204.09006536697035| null|             null|
|    min|R100GMAY1ZSE8B|                 1|               10|                20|    N|                N|
|    max| RZZANCVZNVUP4|                 5|            10020|             10462|    N|                Y|
+-------+--------------+------------------+-----------------+------------------+-----+-----------------+



In [40]:
# Vine Reviews - count of each rating

vine_star_ratings_df = paid_vine_review_df.groupby("star_rating").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "ratings_count")
vine_star_ratings_df.sort("star_rating").show()

+-----------+-------------+
|star_rating|ratings_count|
+-----------+-------------+
|          1|            7|
|          2|            7|
|          3|           43|
|          4|           98|
|          5|          106|
+-----------+-------------+



In [60]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window
vsr_perc_df = vine_star_ratings_df.withColumn('percent', 100 * f.col('ratings_count')/f.sum('ratings_count').over(Window.partitionBy()))
vsr_perc_df.sort("star_rating").show()

+-----------+-------------+------------------+
|star_rating|ratings_count|           percent|
+-----------+-------------+------------------+
|          1|            7| 2.681992337164751|
|          2|            7| 2.681992337164751|
|          3|           43| 16.47509578544061|
|          4|           98|37.547892720306514|
|          5|          106| 40.61302681992337|
+-----------+-------------+------------------+

